In [1]:
import numpy as np
import pandas as pd
import sys
import math
from keras.models import Sequential
from keras.layers import Dense,Dropout ,Activation
from sklearn.svm import SVC

Using TensorFlow backend.


In [2]:
X = pd.read_csv("X_train",low_memory=False)
X_test = pd.read_csv("X_test",low_memory=False)
Y = np.loadtxt("Y_train",dtype=np.int,delimiter=',')
Y_logistic = np.reshape(Y,(-1,1))
Y = Y.flatten()

X=np.array(X,dtype = float)
X_test = np.array(X_test,dtype = float)


X_mean = np.mean(X,axis =0,keepdims = True)
X_std = np.std(X,axis = 0,keepdims = True)

X = (X-X_mean)/X_std
X_test = (X_test-X_mean)/X_std
bias = np.ones((32561,1))
biastest = np.ones((16281,1))
X_logistic = np.concatenate((X,bias),axis = 1)
X_test_logistic = np.concatenate((X_test,biastest),axis = 1)

#bias = np.ones((32561,1))
#biastest = np.ones((16281,1))
#X = np.concatenate((X,bias),axis = 1)
#X_test = np.concatenate((X_test,biastest),axis = 1)
print("X_shape:",X.shape,"\nY_shape:",Y.shape)


X_shape: (32561, 106) 
Y_shape: (32561,)


In [3]:
class logistic_regression():
    def __init__(self):
        pass
    def _init_para(self):
        self.W =np.ones((107,1))
        
    def fit(self, X, Y, valid=None, max_epoch=2000, lr=0.000001):
        assert X.shape[0] == Y.shape[0]
        self._init_para()

        for epoch in range(1, max_epoch+1):
            
            W_grad = -np.dot(X.T,(Y-self.predict(X)))
            #print(W_grad.shape)
            self.W = self.W -(lr*W_grad)
            #print(self.W)

            if epoch%100==0:
                training_loss = self.loss(X,Y)
                #accuracy = self.evaluate(X,Y)
                print('[Epoch%5d] - training loss: %5f'%(epoch, training_loss))
            
    def sigmod(self,z):#   sigmod(z) = 1/(1+e^-z)
        return 1/(1+np.exp(-z))
    
    def predict(self, X, test=False):
        return self.sigmod(np.dot(X,self.W))

    def loss(self, X, Y, pred=None):
        predict_value = self.predict(X)
        return -np.sum(Y*np.log(predict_value+0.00000000001)+(1-Y)*(np.log(1-predict_value+0.00000000001)))


    def evaluate(self, X, Y):
        pred=self.predict(X)
        for i in range(len(pred)):
            if pred[i]>=0.5:
                pred[i] = 1
            else:
                pred[i] =0
            return np.mean(1-np.abs(pred-Y))


In [4]:
logistic_model = logistic_regression()
logistic_model.fit(X_logistic,Y_logistic,max_epoch = 10000)

[Epoch  100] - training loss: 49462.305997
[Epoch  200] - training loss: 34862.255814
[Epoch  300] - training loss: 26246.592140
[Epoch  400] - training loss: 21088.055762
[Epoch  500] - training loss: 18098.741507
[Epoch  600] - training loss: 16220.104454
[Epoch  700] - training loss: 14912.953588
[Epoch  800] - training loss: 13944.184312
[Epoch  900] - training loss: 13174.972544
[Epoch 1000] - training loss: 12560.438753
[Epoch 1100] - training loss: 12080.368998
[Epoch 1200] - training loss: 11695.135036
[Epoch 1300] - training loss: 11387.696303
[Epoch 1400] - training loss: 11150.449163
[Epoch 1500] - training loss: 10960.159789
[Epoch 1600] - training loss: 10804.080743
[Epoch 1700] - training loss: 10677.938866
[Epoch 1800] - training loss: 10577.083859
[Epoch 1900] - training loss: 10502.724193
[Epoch 2000] - training loss: 10461.781170
[Epoch 2100] - training loss: 10440.631333
[Epoch 2200] - training loss: 10426.843834
[Epoch 2300] - training loss: 10416.175234
[Epoch 2400

In [5]:
class Generative_model():
    def __init__(self):
        pass
    
    
    def sigmoid(self,z):#   sigmod(z) = 1/(1+e^-z)
        return 1/(1+np.exp(-z))
    

    def fit(self, X, Y):
        assert X.shape[0] == Y.shape[0]        
        x1 = []#Y=1
        x2 = []#Y=0
        for i in range(X.shape[0]):
            if Y[i]==1:
                x1.append(X[i,:])
            else:
                x2.append(X[i,:])
        x1 = np.array(x1,dtype = float)#~7000*106
        x2 = np.array(x2,dtype = float)#~20000*106
        
        
        self.X_mean_class1 = np.mean(x1,axis =0,keepdims = True)#1*106
        self.X_mean_class2 = np.mean(x2,axis =0,keepdims = True)#1*106

        self.X_len = x1.shape[0]+x2.shape[0]
        self.Pc1 = (x1.shape[0]/X.shape[0])
        self.Pc2 = (x2.shape[0]/X.shape[0])
        
        Sigma1 = np.dot((x1 - self.X_mean_class1).T,(x1 - self.X_mean_class1))/x1.shape[0]
        Sigma2 = np.dot((x2 - self.X_mean_class2).T,(x2 - self.X_mean_class2))/x2.shape[0]
        self.Sigma = self.Pc1*Sigma1 + self.Pc2*Sigma2
        #print("Sigma1:",Sigma1.shape,"\nSigma2:",Sigma2.shape,"\nSigma:",self.Sigma.shape)

    def predict(self, test_X):
        #print("testx:",test_X.shape)
        sigma_inverse = np.linalg.pinv(self.Sigma)
        #print(sigma_inverse)
        w = np.dot((self.X_mean_class1-self.X_mean_class2),sigma_inverse)
        #print("w=",w)
        b = (-0.5) * np.dot(np.dot(self.X_mean_class1, sigma_inverse), self.X_mean_class1.T) + (0.5) * np.dot(np.dot(self.X_mean_class2, sigma_inverse), self.X_mean_class2.T) + np.log(float(self.Pc1)/(self.Pc2))
        z = np.dot(w, X_test.T) + b        
        
        return self.sigmoid(z)

In [6]:
generative_model = Generative_model()
generative_model.fit(X,Y)

In [7]:
dnn_model = Sequential()
dnn_model.add(Dense(50,input_dim = 106))
dnn_model.add(Activation('relu') )
dnn_model.add(Dropout(0.3))
dnn_model.add(Dense(40))
dnn_model.add(Activation('relu') )
dnn_model.add(Dropout(0.2))
dnn_model.add(Dense(30))
dnn_model.add(Activation('relu') )
dnn_model.add(Dropout(0.1))
dnn_model.add(Dense(20))
dnn_model.add(Activation('relu') )
dnn_model.add(Dropout(0.1))
dnn_model.add(Dense(1))
dnn_model.add(Activation('sigmoid'))
dnn_model.compile(loss = 'binary_crossentropy',optimizer= 'adam',metrics=['accuracy'])
dnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                5350      
_________________________________________________________________
activation_1 (Activation)    (None, 50)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 40)                2040      
_________________________________________________________________
activation_2 (Activation)    (None, 40)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 30)                1230      
__________

In [8]:
dnn_model.fit(x = X,y = Y,epochs =30,validation_split= 0.005,verbose = 1)

Train on 32398 samples, validate on 163 samples
Epoch 1/30
32398/32398 [==============================] - 2s 50us/step - loss: 0.3937 - acc: 0.8139 - val_loss: 0.2974 - val_acc: 0.8589
Epoch 2/30
32398/32398 [==============================] - 1s 39us/step - loss: 0.3422 - acc: 0.8417 - val_loss: 0.2893 - val_acc: 0.8528
Epoch 3/30
32398/32398 [==============================] - 1s 37us/step - loss: 0.3295 - acc: 0.8469 - val_loss: 0.2905 - val_acc: 0.8650
Epoch 4/30
32398/32398 [==============================] - 1s 39us/step - loss: 0.3226 - acc: 0.8523 - val_loss: 0.2886 - val_acc: 0.8650
Epoch 5/30
32398/32398 [==============================] - 1s 39us/step - loss: 0.3184 - acc: 0.8550 - val_loss: 0.2739 - val_acc: 0.8589
Epoch 6/30
32398/32398 [==============================] - 1s 39us/step - loss: 0.3149 - acc: 0.8558 - val_loss: 0.2810 - val_acc: 0.8773
Epoch 7/30
32398/32398 [==============================] - 1s 40us/step - loss: 0.3118 - acc: 0.8572 - val_loss: 0.2749 - val_acc: 

In [9]:
svc_model = SVC(kernel='linear', C=1, gamma='auto')
svc_model.fit(X,Y)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [23]:
predict_logistic = logistic_model.predict(X_test_logistic)
predict_dnn = dnn_model.predict(X_test)
predict_generative = generative_model.predict(X_test)
predict_svc =svc_model.predict(X_test)
predict_svc = np.reshape(predict_svc,(-1,1))
predict_logistic = np.reshape(predict_logistic,(-1,1))
predict_generative = np.reshape(predict_generative,(-1,1))
print(predict_logistic.shape,predict_dnn.shape,predict_svc.shape,predict_generative.shape)
finialresult = (predict_logistic*0.2 + predict_dnn*0.6 + predict_generative*0.1 + predict_svc*0.1)
print(finialresult.shape)
for i in range(len(finialresult)):
    if finialresult[i]<0.5:
        finialresult[i] = 0
    else:
        finialresult[i] = 1
finialresult =np.array(finialresult,dtype=int)

(16281, 1) (16281, 1) (16281, 1) (16281, 1)
(16281, 1)


In [26]:
 with open('emsemble.csv', 'w') as f:
        print('id,label', file=f)
        for (i, p) in enumerate(finialresult) :
            print('{},{}'.format(i+1, p[0]), file=f)